In [ ]:
## Experimental Setup

In [1]:
from fastai.vision.all import *
from fastai.distributed import *
# from fastai.metrics import error_rate
# from fastai.callbacks import SaveModelCallback
from fastai.callback.tracker import SaveModelCallback
from fastai.callback.cutmix import CutMix
from fastai.callback.mixup import MixUp

In [2]:
import torch
if torch.cuda.is_available():
    print("Great! Good to go!")
else:
    print('CUDA is not up!')

Great! Good to go!


In [3]:
# from gepcore.utils import cell_graph
# from gepnet.model_v2 import get_gepnet, arch_config
# from gepnet.utils import count_parameters
# torch.backends.cudnn.benchmark = True

from gepcore.utils import cell_graph, convolution
from gepcore.entity import Gene, Chromosome
from gepcore.symbol import PrimitiveSet
from gepnet.model_v1 import get_net, arch_config
from gepnet.utils import count_parameters
torch.backends.cudnn.benchmark = True

In [4]:
from pygraphviz import AGraph
import glob

In [5]:
# comp_graphs = []
# for i in range(3):
#     graph = [AGraph(g) for g in glob.glob('comp_graphs/tevc_2020/{}/*.dot'.format(i))]
#     _, comp_graph = cell_graph.generate_comp_graph(graph)#
#     comp_graphs.append(comp_graph)
#     #print(comp_graph, '\n')

graph = [AGraph(g) for g in glob.glob('comp_graphs/tevc_2020/*.dot')]
_, comp_graphs = cell_graph.generate_comp_graph(graph)#

In [ ]:
# generate new chromosome
# define primitive set
pset = PrimitiveSet('cnn')

# add cellular encoding program symbols
pset.add_program_symbol(cell_graph.end)
pset.add_program_symbol(cell_graph.seq)
pset.add_program_symbol(cell_graph.cpo)
pset.add_program_symbol(cell_graph.cpi)

# add convolutional operations symbols
conv_symbol = convolution.get_symbol()
#pset.add_cell_symbol(conv_symbol.conv3x3)
pset.add_cell_symbol(conv_symbol.sepconv3x3)
#pset.add_cell_symbol(conv_symbol.dilconv3x3)
#pset.add_cell_symbol(conv_symbol.dilconv5x5)
#pset.add_cell_symbol(conv_symbol.sepconv5x5)
#pset.add_cell_symbol(conv_symbol.dwconv3x3)
pset.add_cell_symbol(conv_symbol.maxpool3x3)
#pset.add_cell_symbol(conv_symbol.conv1x1)
#pset.add_cell_symbol(conv_symbol.conv3x1x3)
#pset.add_cell_symbol(conv_symbol.conv1x3)
#pset.add_cell_symbol(conv_symbol.conv3x1)

# comp_graphs = []
# for i in range(3):
#     def gene_gen():
#         return Gene(pset, 2)
#     ch = Chromosome(gene_gen, 3)
#     graph, comp_graph = cell_graph.generate_comp_graph(ch)
#     comp_graphs.append(comp_graph)
#     cell_graph.save_graph(graph, 'comp_graphs/tevc_2020/{}'.format(i))
#     cell_graph.draw_graph(graph, 'comp_graphs/tevc_2020/{}'.format(i))

# for i in range(10):
#     def gene_gen():
#         return Gene(pset, 2)
#     ch = Chromosome(gene_gen, 1)
#     graph, comp_graph = cell_graph.generate_comp_graph(ch)
#     cell_graph.save_graph(graph, 'comp_graphs/ssci_2020/{}'.format(i))
#     cell_graph.draw_graph(graph, 'comp_graphs/ssci_2020/{}'.format(i))

def gene_gen():
    return Gene(pset, 2)
ch = Chromosome(gene_gen, 3)
graph, comp_graphs = cell_graph.generate_comp_graph(ch)
cell_graph.save_graph(graph, 'comp_graphs/tevc_2020')
cell_graph.draw_graph(graph, 'comp_graphs/tevc_2020')

In [6]:
# import random
#seed = 200
# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
# torch.cuda.manual_seed(http://localhost:8888/notebooks/ResearchProjects/geppy_nn/evolution_search.ipynb#seed)
# torch.cuda.manual_seed_all(seed)

# # enable torch backends
# torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
#torch.backends.deterministic = True

In [7]:
conf = arch_config(comp_graphs=comp_graphs,
                   channels=40,
                   repeat_list=[7, 7, 7],
                   classes=10)

net = get_net(conf)
count_parameters(net)

3.29637

In [8]:
#size=32
# cutout_frac = 0.25
# p_cutout = 0.75
# cutout_sz = round(size*cutout_frac)
# cutout_tfm = cutout(n_holes=(2,2), length=(cutout_sz, cutout_sz), p=p_cutout)

path = untar_data(URLs.CIFAR)
# tfms = get_transforms() #xtra_tfms=[cutout()]
# bs = 128
# data = (ImageList.from_folder(path/'train')
#         .split_by_rand_pct(valid_pct=0.1, seed=22) 
#         .label_from_folder()
#         .transform(tfms)
#         .databunch(bs=bs, num_workers=num_cpus())
#         .normalize(cifar_stats))

# dl = ImageDataLoaders(path)
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items=get_image_files,
                   splitter=RandomSplitter(seed=42),
                   get_y=parent_label,
                   batch_tfms=[*aug_transforms(), Normalize.from_stats(*cifar_stats)])

data = dblock.dataloaders(path/'train', bs=64)

In [9]:
# def model_splitter(model):
#     return [params(model)[:132], params(model)[132:]]

def model_splitter(model):
    return [params(model.stem_), params(model.blocks_), params(model.head_)]

In [10]:
# loss_func = nn.CrossEntropyLoss()  loss_func=LabelSmoothingCrossEntropy(), 
cutmix = CutMix()
mixup = MixUp()
save = SaveModelCallback(monitor='accuracy', name='model')
model_dir = '/home/cliff/ResearchProjects/geppy_nn/comp_graphs/tevc_2020'

learn = Learner(data, net, metrics=[error_rate, accuracy],
                cbs=[cutmix, save], model_dir=model_dir)#, splitter=model_splitter) #.load('model')
#learn.to_fp16()   

In [15]:
learn.lr_find()
# learn.recorder.plot()

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 10.91 GiB total capacity; 9.82 GiB already allocated; 18.19 MiB free; 9.82 GiB reserved in total by PyTorch)

In [13]:
# save = SaveModelCallback(learn, monitor='accuracy', name='model_0')
#learn.fit_flat_cos(150, 3e-3, wd=1e-4)
learn.fit_one_cycle(1000, 1e-3) 

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.637959,0.208795,0.063600,0.936400,04:44
1,0.671248,0.195517,0.062900,0.937100,04:43
2,0.659348,0.197513,0.059900,0.940100,04:44
3,0.627469,0.182400,0.057600,0.942400,04:44
4,0.643185,0.179885,0.057000,0.943000,04:43
5,0.754978,0.182333,0.054900,0.945100,04:44
6,0.659895,0.180118,0.054500,0.945500,04:44
7,0.646618,0.174578,0.052000,0.948000,04:41
8,0.657162,0.171490,0.053100,0.946900,04:42
9,0.660536,0.165630,0.052200,0.947800,04:42


Better model found at epoch 0 with accuracy value: 0.9363999962806702.
Better model found at epoch 1 with accuracy value: 0.9370999932289124.
Better model found at epoch 2 with accuracy value: 0.9401000142097473.
Better model found at epoch 3 with accuracy value: 0.9423999786376953.
Better model found at epoch 4 with accuracy value: 0.9430000185966492.
Better model found at epoch 5 with accuracy value: 0.9451000094413757.
Better model found at epoch 6 with accuracy value: 0.9455000162124634.
Better model found at epoch 7 with accuracy value: 0.9480000138282776.
Better model found at epoch 12 with accuracy value: 0.9491999745368958.
Better model found at epoch 14 with accuracy value: 0.9496999979019165.
Better model found at epoch 18 with accuracy value: 0.949999988079071.
Better model found at epoch 20 with accuracy value: 0.9510999917984009.
Better model found at epoch 21 with accuracy value: 0.9513999819755554.
Better model found at epoch 23 with accuracy value: 0.9527999758720398.
B

IndexError: list index out of range

In [ ]:
####################### Testing ###########################

In [ ]:
# graph_dir = 'comp_graphs/new/*.dot' #'/content/mnt/My Drive/Colab Notebooks/ResearchProject/comp_graphs/experiment_1/best/indv_1/*.dot'
# graph = [AGraph(g) for g in glob.glob(graph_dir)]
# _, comp_graph = cell_graph.generate_comp_graph(graph)

# comp_graphs = []
# for i in range(3):
#     graph = [AGraph(g) for g in glob.glob('comp_graphs/new/{}/*.dot'.format(i))]
#     _, comp_graph = cell_graph.generate_comp_graph(graph)#
#     comp_graphs.append(comp_graph)

graph = [AGraph(g) for g in glob.glob('comp_graphs/tevc_2020/*.dot')]
_, comp_graphs = cell_graph.generate_comp_graph(graph)#

In [ ]:
conf = arch_config(comp_graphs=comp_graphs,
                   channels=40,
                   repeat_list=[6, 6, 6],
                   classes=10)

net = get_net(conf)
count_parameters(net)

In [ ]:
# tfms = get_transforms(do_flip=False)
path = untar_data(URLs.CIFAR)
# bs = 256

# data = (ImageList.from_folder(path)
#         .split_by_folder(train='train', valid='test')
#         .label_from_folder()
#         .transform(tfms, size=32)
#         .databunch(bs=bs, num_workers=num_cpus())
#         .normalize(cifar_stats))

dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items=get_image_files,
                   splitter=GrandparentSplitter(train_name='train', valid_name='test'),
                   get_y=parent_label,
                   batch_tfms=[*aug_transforms(do_flip=False), Normalize.from_stats(*cifar_stats)])

data = dblock.dataloaders(path, bs=128)

In [ ]:
#model_dir = '/home/cliff/ResearchProjects/geppy_nn/comp_graphs/new' 
#'/content/mnt/My Drive/Colab Notebooks/ResearchProject/models/'
#model = Learner(data, net, metrics=[accuracy, error_rate]).load(model_dir+'/model_1')
#model.validate()
model_dir = '/home/cliff/ResearchProjects/geppy_nn/comp_graphs/tevc_2020'
model = Learner(data, net, metrics=[error_rate, accuracy]).load(model_dir+'/model')
model.validate()

In [ ]:
preds, targets = model.tta()

In [ ]:
accuracy(preds, targets).item()

In [ ]:
preds,y,losses = model.get_preds(ds_type=DatasetType.Valid, with_loss=True)
preds[:5]

In [1]:
def duration(sec):
    days, sec = sec//(24*3600), sec%(24*3600)
    hrs, sec = sec//(3600), sec%3600
    mins, sec = sec//60, sec%60
    print()
    print('Duration: %d:%02d:%02d:%02d' %(days, hrs, mins, sec))

In [5]:
duration(22*100*20)


Duration: 0:12:13:20
